# Agentic AI Workshop

# Introducing: The Price Is Right

We are going to build a multi-agent framework that will:
1. Pull RSS feeds from the web and interpret them, looking for promising deals (multi-shot prompting, structured outputs)
2. Use RAG and a Frontier Model to estimate the price of a product, using a knowledge base
3. Use a QLoRA fine tuned LLM to estimate the price
4. Send a push notification when it finds a deal that's worth significantly less than it estimates
5. A planning agent will orchestrate this

If you're having any set up problems, please see the [troubleshooting](../troubleshooting.ipynb) notebook in the parent directory.  
For more details on using Jupyter Lab, please see the [Guide To Jupyter](../Guide%20to%20Jupyter.ipynb) notebook.  
If you'd like to brush up your Python skills, there's an [Intermediate Python](../Intermediate%20Python.ipynb) notebook.


## First: Let's look at the Agent code for interpreting RSS feeds, including figuring out the pricing, using multi-shot prompting

## Remember: just get some intuition for this, and run it yourself later..

In [1]:
# Some imports

from dotenv import load_dotenv
from price_agents.deals import ScrapedDeal
from openai import OpenAI
import logging

In [2]:
# Set up our env variables

load_dotenv(override=True)
openai = OpenAI()
MODEL = "gpt-5-mini"

In [3]:
# I wrote this code to fetch RSS feeds

scraped_deals = ScrapedDeal.fetch(show_progress=True)

100%|█████████████████████████████████████████████| 3/3 [00:07<00:00,  2.63s/it]


In [4]:
print(scraped_deals[2].describe())

Title: Sony X90L Series XR65X90L 65" 4K HDR 120Hz LED UHD Smart TV for $898 + free shipping
Details: That's the best price we've ever seen. You'd pay over $1,000 at most stores. Buy Now at Walmart
Features: 3840x2160 (4K) resolution HDR 10 120Hz refresh rate Google TV compatible with Alexa, Google Assistant four HDMI ports Model: XR65X90L UPC: 027242927438
URL: https://www.dealnews.com/products/Sony-Electronics/Sony-X90-L-Series-XR65-X90-L-65-4-K-HDR-120-Hz-LED-UHD-Smart-TV/445744.html?iref=rss-c142


## We are going to ask GPT-4.1-mini to summarize deals and identify their price

In [5]:
SYSTEM_PROMPT = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 
    
{
    "deals": [
        {
            "product_description": "Your clearly expressed summary of the product in 3-4 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
            "price": 99.99,
            "url": "the url as provided"
        },
        ...
    ]
}"""

In [6]:
USER_PROMPT_PREFIX = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price that is greater than 0.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 
    
Deals:
    
"""

USER_PROMPT_SUFFIX = """\n\nStrictly respond in JSON and include exactly 5 deals, no more.
Your response must follow this format:

{
    "deals": [
        {
            "product_description": "Your summary of the product in 3-4 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
            "price": 99.99,
            "url": "the url as provided"
        },
        {
            "product_description": "Your summary of the product in 3-4 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
            "price": 210.30,
            "url": "the url as provided"
        },
        ...
]}
"""

In [7]:
# this makes a suitable user prompt given scraped deals

def make_user_prompt(scraped):
    user_prompt = USER_PROMPT_PREFIX
    user_prompt += '\n\n'.join([scrape.describe() for scrape in scraped])
    user_prompt += USER_PROMPT_SUFFIX
    return user_prompt

In [8]:
# Let's create a user prompt for the deals we just scraped, and look at how it begins

user_prompt = make_user_prompt(scraped_deals)
print(user_prompt[:2000])

Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price that is greater than 0.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

Title: TV Deals at Costco: Extra $200 off $999 + free shipping
Details: Save on TVs from Samsung, LG, TCL, and more. We've pictured the Samsung The Frame Series QN55LS03FADXZA 55" QLED 4K HDR Smart TV for $1,000 after the in-cart discount. Buy Now at Costco
Features: 
URL: https://www.dealnews.com/TV-Deals-at-Costco-Extra-200-off-999-free-shipping/21

## Calling OpenAI using their Python client

Recall the format of OpenAI calls:

When you call OpenAI, you pass in your conversation as a list of python dictionaries:

```
    [
        {"role": "system", "content": "system_message_goes_here"},
        {"role": "user", "content": "user_prompt_goes_here"}
    ]
```

In [9]:
# Create the user prompt that reflects the deals we have scraped

user_prompt = make_user_prompt(scraped_deals)

# Call OpenAI using their Python client. Request a response in JSON and stream it back.

stream = openai.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_prompt}
  ],
    stream=True,
    response_format={"type": "json_object"}
)

response = ""
for chunk in stream:
    chunk_text = chunk.choices[0].delta.content or ''
    response += chunk_text
    print(chunk_text, end="", flush=True)

{
  "deals": [
    {
      "product_description": "The Sony X90L is a 65-inch 4K LED UHD smart television that delivers HDR10 picture quality and a smooth 120Hz refresh rate for fast-moving content. It runs Google TV and supports voice control via Alexa and Google Assistant, providing broad streaming app compatibility and integrated smart features. The set includes four HDMI ports for multiple source connections and is identified by model XR65X90L for easy reference. This TV targets users who want a high-refresh, high-resolution panel with modern smart-TV functionality.",
      "price": 898,
      "url": "https://www.dealnews.com/products/Sony-Electronics/Sony-X90-L-Series-XR65-X90-L-65-4-K-HDR-120-Hz-LED-UHD-Smart-TV/445744.html?iref=rss-c142"
    },
    {
      "product_description": "The Mloong 4th-Gen Ryzen 5 gaming desktop is built around a 4th Generation AMD Ryzen 5 5500U 2.1GHz six-core processor and ships with 16GB of RAM and a 1TB SSD for fast storage and multitasking. It runs

In [10]:
import json
json.loads(response)

{'deals': [{'product_description': 'The Sony X90L is a 65-inch 4K LED UHD smart television that delivers HDR10 picture quality and a smooth 120Hz refresh rate for fast-moving content. It runs Google TV and supports voice control via Alexa and Google Assistant, providing broad streaming app compatibility and integrated smart features. The set includes four HDMI ports for multiple source connections and is identified by model XR65X90L for easy reference. This TV targets users who want a high-refresh, high-resolution panel with modern smart-TV functionality.',
   'price': 898,
   'url': 'https://www.dealnews.com/products/Sony-Electronics/Sony-X90-L-Series-XR65-X90-L-65-4-K-HDR-120-Hz-LED-UHD-Smart-TV/445744.html?iref=rss-c142'},
  {'product_description': 'The Mloong 4th-Gen Ryzen 5 gaming desktop is built around a 4th Generation AMD Ryzen 5 5500U 2.1GHz six-core processor and ships with 16GB of RAM and a 1TB SSD for fast storage and multitasking. It runs Windows 11 Home and is configured 

# Putting this into an "Agent"

I've packaged this code into a class called `ScannerAgent`

There are various Agent frameworks that add an abstraction layer, but in our case it's easy just to write the code directly.

In [11]:
root = logging.getLogger()
root.setLevel(logging.INFO)

In [12]:
from price_agents.scanner_agent import ScannerAgent

scanner = ScannerAgent()
scanner.scan()

INFO:root:[Scanner Agent] Scanner Agent is initializing
INFO:root:[Scanner Agent] Scanner Agent is ready
INFO:root:[Scanner Agent] Scanner Agent is about to fetch deals from RSS feed
INFO:root:[Scanner Agent] Scanner Agent received 15 deals not already scraped
INFO:root:[Scanner Agent] Scanner Agent is calling OpenAI using Structured Output
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[Scanner Agent] Scanner Agent received 5 selected deals with price>0 from OpenAI


DealSelection(deals=[Deal(product_description="Sony X90L Series XR65X90L is a 65-inch 4K LED TV with 3840x2160 resolution and HDR10 support, delivering crisp visuals and high dynamic range. It features a 120Hz refresh rate for smoother motion in games and fast-paced content, and runs Google TV with built-in Google Assistant and Alexa compatibility. The set includes four HDMI ports for multiple device connections and uses Sony's XR processing to enhance contrast and color accuracy.", price=898.0, url='https://www.dealnews.com/products/Sony-Electronics/Sony-X90-L-Series-XR65-X90-L-65-4-K-HDR-120-Hz-LED-UHD-Smart-TV/445744.html?iref=rss-c142'), Deal(product_description='Mloong 4th-Gen Ryzen 5 gaming desktop is built around an AMD Ryzen 5 5500U 6-core CPU (base 2.1GHz) and ships with 16GB of RAM and a 1TB SSD for responsive multitasking and fast storage. It runs Windows 11 Home and is configured for gaming and general productivity out of the box. The system is positioned as an affordable p